# Lago Workshop!
Introduction to Lago CLI and SDK.

Lab 0
=====

**Goal: Get your machine setup for running the lab**

Clone the workshop repository from GitHub:
```bash

$ git clone https://github.com/lago-project/lago-workshop.git
```

Switch to the workshop virtual environment by running:

```bash
$ workon lago_workshop
```

Lab A
=====

** Goal: Get to know Lago Command Line **

Ensure Lago is working on your laptop by running:
```bash
$ lago --version
```

Change to your to the lago-workshop first lab example:

```bash
$ cd ~/lago-workshop/intro
```

Explore the init file by running:
```bash
$ cat init-simple.yaml
```

Initalize the virtual environment:
```bash
$ lago init init-simple.yaml
```

Start the enviornment:
```bash
$ lago start
```

When the enviornment is up, run:
```bash
$ lago status
```
notice that the VM ``vm-01`` status should be "running".

Now we will run a command inside the VM with SSH, we will check the VM's uptime:
```bash
$ lago shell vm-01 -c 'uptime'
```

Now lets see where Lago stores metadata, run:

```bash
$ tree .lago
```

Exercise
*********
Interactive shell can be done by running ``lago shell vm-01``, this is parallel to to "normal" SSH you would do to a remote machine. Log into the machine and extract which distribution it is, normally this information is stored in a file with the glob matching ``/etc/*release*``. To exit the interactive shell run ``exit``.





<details>
  <summary>Click here for for answer</summary>
  Thats right, it is CentOS 7.3!, this information can be extracted by SSH'ing into the VM and running cat /etc/*release*.
</details>



Finally, lets destroy the enviornment, run inside the same directory:
```bash
$ lago destroy --yes
```




Lab B
=====
**Goal: Working with Lago SDK!**

Goal: Initalize the Lago enviornment, directly from Python.

Change to the workshop intro directory:
```bash
$ cd ~/lago-workshop/intro
```

Start an interactive IPython session by running:
```bash
$ ipython
```

Inside the IPython console, first import the dependencies:

In [1]:
import logging
from lago import sdk

Now create the same environment by calling ``sdk.init``, also we'll setup logging for debugging at ``/tmp/lago.log``:

In [2]:
env = sdk.init(config='init-simple.yaml',
              workdir='/tmp/lago_sdk_simple_example',
              loglevel=logging.DEBUG,
              log_fname='/tmp/lago.log')


Start the environment:

In [3]:
env.start()

Check which VMs are available by using the ``get_vms()`` method:

In [4]:
vms = env.get_vms()
print vms

{'vm-01': <lago.vm.DefaultVM object at 0x7f5e46e74490>}


In [5]:
vm = vms['vm-01']

Now we'll print some metadata available in the virtual machine object, namely the VM's IP and Linux distribution:

In [6]:
distro = vm.distro()
print distro
ip = vm.ip()
print ip

el7
192.168.202.2


By running ``dir``, we can see other methods which are available in the VM object:

In [14]:
dir(vm)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__metaclass__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_artifact_paths',
 '_detect_service_provider',
 '_get_service_provider',
 '_get_vm_provider',
 '_normalize_spec',
 '_scp',
 '_service_class',
 '_spec',
 '_ssh_client',
 '_template_metadata',
 'alive',
 'all_ips',
 'bootstrap',
 'collect_artifacts',
 'copy_from',
 'copy_to',
 'cpu_model',
 'cpu_vendor',
 'create_snapshot',
 'defined',
 'disks',
 'distro',
 'export_disks',
 'extract_paths',
 'groups',
 'guest_agent',
 'has_guest_agent',
 'interactive_console',
 'interactive_ssh',
 'ip',
 'iscsi_name',
 'metadata',
 'mgmt_name',
 'mgmt_net',
 'name',
 'nets',
 'nics',
 'provider',
 'reboot',
 

Now lets run an automated SSH command to the VM:

In [7]:
res = vm.ssh(['hostname', '-f'])

The returned object of type ``CommandStatus`` includes 3 variabiles: ``out``, ``err`` which contain the SSH command standard output, standard error and finally the ``code`` which is the command's exit code on the VM.

In [8]:
res

CommandStatus(code=0, out='vm-01.lago.local\n', err='')

*Exercise* : Try running a command that will not work, and check what is the error code. 

*Exercise* : For the brave(on the whiteboard)

Another important functionality of Lago, is to collect existing files from the virtual machine. This is the function signature:
```python
    def collect_artifacts(self, output_dir, ignore_nopath):
```
The ``output_dir`` parameter, is where you would like to collect those files. The files to collect are defined in the LagoInitFile, if not defined it will default to ``/var/log``.
*Note that if the output_dir already exists it will try to rotate it, so never do it on your home directory. This is designed that way so you could collect results of several different runs to the same directory*

First open another terminal window, with that in hand run:


In [9]:
env.collect_artifacts(output_dir='/tmp/myluckyday', ignore_nopath=True)

Now on your other terminal window, run:
```bash
$ tree /tmp/myluckyday
```
You should see a nice structutre of directories composed of the VM name, and the path which was collected.
Now run the same command again, and check out the ``/tmp/myluckday`` directory in your terminal, you should see that logs were collected twice. 

Now lets see how you can reload the environment to a different object, and by that perserve it states. Here we will load it into the same session, but you could have exited and reloaded it later, run:

In [11]:
new_env = sdk.load_env(workdir='/tmp/lago_sdk_simple_example')

In [12]:
new_env.get_vms()

{u'vm-01': <lago.vm.DefaultVM at 0x7f5e46c7f210>}

Finally, lets destroy the environment, leaving no traces: 

In [13]:
new_env.destroy()